In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:

import re

def parse_prediction(predicted_text_list,unit_list):

    pred_list=[]

    for i in range(len(predicted_text_list)):
        text=predicted_text_list[i]
        unit=unit_list[i]




        # 正規表現パターンを定義: "is" の後に数値部分 (-?(\d+(\.\d+)?))
        pattern = r"is\s*(-?\d+(\.\d+)?)"

        # 正規表現によるマッチング
        match = re.search(pattern, text)
        if match:
            predict=(match.group())
            #predict=predict.replace(unit,"").strip()
            #predict=predict.replace(unit[1:-1],"").strip()
            predict=predict.replace("is ","").strip()
            try:
                pred_val=float(predict)
            except:
                pred_val=None
        else:
            pred_val=None

        if pred_val is None:
            #正規表現で､「数値」 + unitを抽出｡数値は小数点やマイナスも含む｡unitの前にスペースがある場合もない場合もある
            unit_pattern = re.escape(unit).replace(r'\[', r'\[?').replace(r'\]', r'\]?')
            pattern = rf"-?\d+(\.\d+)?\s*{unit_pattern}"
            # 正規表現によるマッチング
            match = re.search(pattern, text)
            if match:
                predict=(match.group())
                predict=predict.replace(unit,"").strip()
                predict=predict.replace(unit[1:-1],"").strip()
                try:
                    pred_val=float(predict)
                except:
                    pred_val=None
        pred_list.append(pred_val)
    return pred_list

In [ ]:
import glob
from tqdm import tqdm
json_list=[]

json_list+=glob.glob("eval_results_1002/*.json")
json_list+=glob.glob("eval_results_1002/*.json.gz")

mode="test"

error_rate_dict={}
count_dict={}
json_list.sort()

total_view=3
cnt=0
for target_json in tqdm(json_list):
    cnt+=1



    #if target_json.find(mode)==-1:
    #    continue

    df=pd.read_json(target_json)
    #predicted_valueをfloatに変更｡変換できないものはNaNにする
    df["predicted_value_float"] = pd.to_numeric(df["predicted_value"], errors='coerce')

    if target_json.find("meta")==-1:
        predicted_text_list=df["predicted_text"].tolist()
        unit_list=df["unit"].tolist()
        predicted_value_list=parse_prediction(predicted_text_list,unit_list)

        #df内がnanの場合は、predicted_value_listを代入
        for i in range(len(predicted_value_list)):
            if pd.isnull(df["predicted_value_float"][i]):
                #df["predicted_value_float"][i]=predicted_value_list[i]
                df.loc[i, "predicted_value_float"]=predicted_value_list[i]
        #df["predicted_value_float"]=predicted_value_list


    #actual_value, predicted_valueの誤差率を計算(絶対値を取る
    df["error_rate"]=(df["Value"]-df["predicted_value_float"])/df["Value"]
    df["error_rate"]=df["error_rate"].abs()

    #error

    df["error"]=-(df["Value"]-df["predicted_value_float"])

    #誤差率が0以上のものを抽出
    sel_df=df[df["error_rate"]>=0]
    sel_df["predicted_text"].tolist()

    #plt.figure()
    #plt.title(target_json)
    #sns.boxplot(x="error_rate",y="Property",data=sel_df)
    #plt.xlim(-1,2)
    #xはlog
    #plt.xscale("log")
    #x=0.1に縦線を引く
    #plt.axvline(x=0.1, color='r', linestyle='--')
    if cnt<total_view:
        plt.figure()
        plt.title(target_json[-50:]+"_"+str(sel_df.shape[0]))
        #sns.histplot(data=sel_df,x="error_rate",bins=100,hue="Property")
        ax=sns.histplot(data=sel_df[sel_df["error_rate"]<1],x="error_rate",
                    bins=10,hue="Property",
                    multiple="stack"
                    )
        #plt.xlim(0,1)
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    error_rate_dict[target_json]=sel_df["error_rate"].median()
    count_dict[target_json]=sel_df.shape[0]

In [ ]:
error_rate_df=pd.DataFrame(error_rate_dict.items(),columns=["json","median_error_rate"])
count_df=pd.DataFrame(count_dict.items(),columns=["json","count"])
error_rate_df=error_rate_df.merge(count_df,on="json")
error_rate_df

In [ ]:
error_rate_df["temp"]=error_rate_df["json"].str.split("checkpoint-").str[-1].str.split(".").str[0]
error_rate_df["step"]=error_rate_df["temp"].str.split("_").str[0]
error_rate_df["step"] = pd.to_numeric(error_rate_df["step"], errors='coerce')
error_rate_df["step"] = error_rate_df["step"].fillna(0)
#error_rate_df["type"]=error_rate_df["temp"].str.split("_").str[1]
error_rate_df["type"]=error_rate_df["json"].str.split("_").str[-1].str.split(".").str[0]
error_rate_df
#sns.pointplot(data=error_rate_df,x="step",y="median_error_rate",hue="type")

In [ ]:
aug_const=9600

spl=error_rate_df["json"].str.split("brad-").str[-1].str.split("-zero1").str[0].str.split("_")
error_rate_df["Trained records"]=(spl.str[0].astype(float))
error_rate_df["Trained records per fact"]=error_rate_df["Trained records"]/aug_const
error_rate_df["PMC records"]=spl.str[1].astype(int)
error_rate_df

In [ ]:
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
sel_df=error_rate_df[error_rate_df["PMC records"]==20000]
plt.figure()
# Lineplotを描画
ax = sns.lineplot(data=sel_df,
                  x="Trained records per fact", y="median_error_rate",
                  hue="type", palette="viridis", alpha=0.7)

# Scatterplotを重ねる
sns.scatterplot(data=sel_df,
                x="Trained records per fact", y="median_error_rate",
                hue="type", palette="viridis", alpha=0.7, legend=False, ax=ax)

# 凡例を移動
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# タイトルを追加
#plt.title(mode)

# グラフを表示
plt.show()


In [ ]:
#60万ではなく50万だった
for mode in ["train","test"]:
    sel_df=error_rate_df[error_rate_df["type"]==mode]
    plt.figure()
    # Lineplotを描画
    ax = sns.lineplot(data=sel_df,
                    x="Trained records per fact", y="median_error_rate",
                    hue="PMC records", palette="viridis", alpha=0.7)

    # Scatterplotを重ねる
    sns.scatterplot(data=sel_df,
                    x="Trained records per fact", y="median_error_rate",
                    hue="PMC records", palette="viridis", alpha=0.7, legend=False, ax=ax)

    # 凡例を移動
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    # タイトルを追加
    plt.title(mode)

    # グラフを表示
    plt.show()

In [ ]:
error_rate_df

In [ ]:
import copy


for mode in ["test","train"]:
    plt.figure()
    ax=sns.pointplot(data=error_rate_df[error_rate_df["type"]==mode],
                     x="PMC records",y="median_error_rate",
                     hue="Trained records per fact",
                     palette="viridis",alpha=0.7)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.title(mode)

    plt.figure()
    ax=sns.pointplot(data=error_rate_df[error_rate_df["type"]==mode],
                     x="PMC records",y="count",
                     hue="Trained records per fact",
                     palette="viridis",alpha=0.7)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.title(mode)

In [ ]:
error_rate_df